In [1]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2

In [14]:
inds = ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth',
       'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry',
       'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper',
       'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
inds=['Beer', 'Hlth', 'Fin','Rtail','Whlsl']
ind_rets=erk.get_ind_returns(weighting="vw", n_inds=49)["2014":"2018"] 
ind_mktcap=erk.get_ind_market_caps(n_inds=49,weights=False)["2014":"2018"]
cov=ind_rets.cov() 

In [6]:
def risk_contribution(w,cov):
    '''
    Compute the contributions to risk of the constituents of the portfolio
    '''
    total_portfolio_var = erk.portfolio_vol(w,cov)**2 #variance, hence squaring
    marginal_contrib = cov@w
    risk_contrib = np.multiply(marginal_contrib,w.T)/total_portfolio_var
    return risk_contrib

In [20]:
cw=erk.risk_contribution(erk.weight_cw(ind_rets,ind_mktcap),cov) 
cw = cw.sort_values(ascending=False)*100
cw.head(5).round(2)

Banks    10.40
Softw     9.84
Drugs     8.91
Oil       8.52
Rtail     6.72
dtype: float64

In [21]:
#Difference between largest and smallest risk 
(cw.values[0]-cw.values[-1]).round(2)


10.39

In [39]:
ew = (erk.risk_contribution(erk.weight_ew(ind_rets),cov))
ew = ew.sort_values(ascending=False)*100
ew.head().round(2)

Steel    3.09
Ships    3.00
Coal     2.90
Mines    2.86
BldMt    2.81
dtype: float64

In [40]:
#Difference between largest and smallest risk

(ew.values[0]-ew.values[-1]).round(2)


2.5

In [42]:
from scipy.optimize import minimize

def target_risk_contributions(target_risk,cov):
    """
    Returns the weights of the portfolio that gives you the weights such
    that the contributions to portfolio risk are as close as possible to
    the target_risk, given the covariance matrix
    """
    n = cov.shape[0]
    init_guess = np.repeat(1/n,n)
    bounds = ((0.0,1.0),)*n

    weights_sum_to_1 = {
        'type':'eq',
        'fun':lambda weights:np.sum(weights)-1
    }

    def msd_risk(weights,target_risk,cov):
        '''
        Returns the mean squared difference between the risk contributions
        between the weights and target risk
        '''
        w_contribs = risk_contribution(weights,cov)
        #Difference in square difference for each stock
        #and then take the sum
        return ((w_contribs-target_risk)**2).sum()

    weights = minimize(
        msd_risk,init_guess,
        args=(target_risk,cov),method='SLSQP',
        options={'disp':False},
        constraints=(weights_sum_to_1),
        bounds=bounds
    )
    return weights.x

def equal_risk_contributions(cov):
    n = cov.shape[0]
    #I want my risk contributions equally for all portfolio stocks
    return target_risk_contributions(target_risk=np.repeat(1/n,n),cov=cov)

def weight_erc(r,cov_estimator=erk.sample_cov,**kwargs):
    est_cov = cov_estimator(r,**kwargs) #will just return sample covariance
    # print(est_cov.columns)
    return equal_risk_contributions(est_cov)

In [43]:
(risk_contribution(weight_erc(ind_rets,erk.sample_cov),cov).sort_values(ascending=False)*100).head().round(2)

Hshld    2.05
Beer     2.05
Other    2.05
RlEst    2.05
Txtls    2.05
dtype: float64

In [44]:
(risk_contribution(weight_erc(ind_rets,erk.sample_cov),cov).sort_values(ascending=False)*100).tail().round(2)

Drugs    2.03
Mines    2.03
Ships    2.03
Steel    2.03
Util     2.02
dtype: float64

In [48]:
print((weight_erc(ind_rets,erk.sample_cov)*100).round(2).max())
weights = weight_erc(ind_rets,erk.sample_cov).round(2)
cov.columns[list(weights).index(weights.max())]

5.22


'Util'

In [49]:
print((weight_erc(ind_rets,erk.sample_cov)*100).round(2).min())

cov.columns[list(weights).index(weights.min())]

1.28


'Chems'